In [1]:
import os
import fitz
import time
import numpy as np

In [2]:
class base:
    def __init__(self):
        self.wherepdf='D:\\[Keep]\\Desktop\\project\\pdfcollector'
        self.placefile='D:\\[Keep]\\Desktop\\project'
        self.filename='total'

In [3]:
class file(base):
    def filenames(self)->list:
        '''列出資料夾中有的files'''
        files=os.listdir(self.wherepdf)
        for i in range(len(files)):
            files[i]=files[i].replace('.pdf','')
        return files
    def findpdfPutpath(self,pdfname:str)->str:
        '''findpdfpath(pdfname檔案名字)'''
        pdfcorrect=str(self.wherepdf+'\\'+pdfname+'.pdf')
        return pdfcorrect
    def findtxtPutpath(self,txtname:str)->str:
        '''findtxtpath(txtname檔案名字)'''
        pdfcorrect=str(self.placefile+'\\'+self.filename+'\\'+txtname+'.txt')
        return pdfcorrect
    def lenoftxt(self,filepath:str)->str:
        '''lenoftxt(txt檔路徑filepath)'''
        try:
            txt=open(filepath,'r',encoding='utf-8')
            txtlen=len(txt.read())
            txt.close()
            return txtlen  
        except:
            print('cant find txt')
    def createdetail(self,name:list,processtime:list,pdflens:list):
        '''傳入list createdetail(self,名字name,時間processtime,長度pdflens)'''
        try:
            detail=open(self.placefile+'\\'+'PDF_process_detail.txt','w',encoding='utf-8')
            for i in range(len(name)):
                detail.writelines(str(name[i])+"Time:  "+str(processtime[i])+"秒   "+"Len:  "+str(pdflens[i])+'個字  \n')
            detail.close()
        except:
            print('cant create index')
    def Pdftotxtfile(self,txtstring:str,txtpath:str):
        '''writepdftotxt(self,全部字串txtstring,要放的路徑txtpath)'''
        try:
            txt=open(txtpath,'w',encoding='utf-8')
            txt.write(txtstring)
            txt.close()
        except:
            print('cant create index')
    def  createPuttxtdir(self):
        '''創建要放txt的file'''
        os.chdir(self.placefile+'\\'+self.filename)
        if  not os.path.isdir(self.placefile+'\\'+self.filename):
            os.mkdir(self.placefile+'\\'+self.filename)
            
            
            

In [4]:
class fitztry(base):
    def getpdflinesinfo(self,pdfpath):
        '''getpdflinesinfo(self,pdfpath)輸入pdf的path'''
        doc = fitz.open(pdfpath)
        total=[]
        count=0
        for i in doc:
            total=total+i.get_text('blocks')
        for j in range(len(total)):
            total[j-count]=list(total[j-count])
            total[j-count][4]=total[j-count][4].replace("\n", "")
            total[j-count][4]=total[j-count][4].strip(" ")
            if(total[j-count][4]==''):
                total.pop(j-count)
                count=count+1
        return total

In [24]:
class stringprocess(base):
    def x0list(self,totallist):
        '''輸入getpdflinesinfo之矩陣即可以找出x0的位置形成list'''
        x0list=[]
        for i in totallist:
            x0list.append(i[0])
        return x0list
    def x1list(self,totallist:list):
        '''輸入getpdflinesinfo之矩陣即可以找出x0的位置形成list'''
        x1list=[]
        for i in totallist:
            x1list.append(i[2])
        return x1list
    def y0list(self,totallist:list):
        '''輸入getpdflinesinfo之矩陣即可以找出x0的位置形成list'''
        y0list=[]
        for i in totallist:
            y0list.append(i[1])
        return y0list
    def y1list(self,totallist:list):
        '''輸入getpdflinesinfo之矩陣即可以找出x0的位置形成list'''
        y1list=[]
        for i in totallist:
            y1list.append(i[3])
        return y1list
    def quantile(self,totallist):
        '''輸出上下四分位距值'''
        quantilelow=np.quantile(totallist,0.25,interpolation='lower')
        quantilehigh=np.quantile(totallist,0.75,interpolation='higher')
        return quantilelow,quantilehigh
    def std(self,totallist):
        '''傳回矩陣標準差'''
        stdd=np.std(totallist, ddof=1)
        return stdd
    def changearticlelines(self,totallist,quantilex0low:float,quantilex1high:float):
        '''修改靠左對齊且右方提前結束的換行,英文字在txt中自動換行'''
        totallistt=totallist
        for i in range(len(totallistt)):
            if ((quantilex0low-1)<totallistt[i][0]<(quantilex0low+1))& (not ((quantilex1high-6)<totallistt[i][2]<(quantilex1high+6))):
                totallistt[i][4]=str(totallistt[i][4])+'\n'
                #print(totallistt[i][4])
        return totallistt
    def changespeciallines(self,totallist:list,quantilex0low:float,quantilex1high:float):
        '''修改沒靠左對齊且右方提前結束的換行'''
        totallistt=totallist
        for i in range(len(totallistt)):
            if (not((quantilex0low-1)<totallistt[i][0]<(quantilex0low+1)))& (not ((quantilex1high-7)<totallistt[i][2]<(quantilex1high+7))):
                totallistt[i][4]=str(totallistt[i][4])+'\n'
                #print(totallistt[i][4])
        return totallistt
    def deletepictureinfo(self,totallist:list):
        '''刪除image訊息不已後方判斷'''
        totallistt=totallist
        count=0
        for i in range(len(totallistt)):
            try:
                if  (list(totallistt[i-count][4])[0]=='<')&(list(totallistt[i-count][4])[1]=='i')&(list(totallistt[i-count][4])[2]=='m'):
                    #print(totallistt[i-count][4])
                    totallistt.pop(i-count)            
                    #print(count)
                    count=count+1
            except:
                continue
        return totallistt
    def deletefront(self,totallist:list):
        totallistt=totallist
        count=0
        for i in range(len(totallistt)):
            if  (float(totallistt[i-count][1])<50)or(float(totallistt[i-count][3])>770):
                #print(totallistt[i-count])
                totallistt.pop(i-count)                
                count=count+1
        return totallistt

        


In [25]:
import time
filee=file()
fitztryy=fitztry()
stringprocesss=stringprocess()
timeuse=[]
lenoftxt=[]
for i in filee.filenames():
    timefirst=time.time()
    k=filee.findpdfPutpath(i)
    k=fitztryy.getpdflinesinfo(k)
    x0list=np.sort(stringprocesss.x0list(k))
    x1list=np.sort(stringprocesss.x1list(k))
    quantilelow,m=stringprocesss.quantile(x0list)
    n,quantilehigh=stringprocesss.quantile(x1list)
    k=stringprocesss.changearticlelines(k,quantilelow,quantilehigh)
    k=stringprocesss.changespeciallines(k,quantilelow,quantilehigh)
    k=stringprocesss.deletepictureinfo(k)
    k=stringprocesss.deletefront(k)
    totalstring=''
    for j in range(len(k)):
        totalstring=totalstring+str(k[j][4])
    wheretoputtxt=filee.findtxtPutpath(i)
    timesecond=time.time()
    filee.createPuttxtdir()
    filee.Pdftotxtfile(totalstring,wheretoputtxt)
    #print(wheretoputtxt)
    r=filee.lenoftxt(wheretoputtxt)
    e=timesecond-timefirst
    timeuse.append(r)
    lenoftxt.append(e)
filee.createdetail(filee.filenames(),lenoftxt,timeuse)

In [7]:
r=filee.lenoftxt('D:\\[Keep]\\Desktop\\project\\total\\碩士論文_盧毅_v1.7_20200819.txt')
a=open('D:\\[Keep]\\Desktop\\project\\total\\fb220309225809.txt',encoding='utf-8')
filee.createdetail(1,2,3)

cant create index


測試區

In [8]:
p=fitztry()
y=stringprocess()
a=p.getpdflinesinfo('D:\\[Keep]\\Desktop\\project\\pdfcollector\\fb220309225809.pdf')
o=y.x0list(a)
quantilex0low,m=y.quantile(o)
r=y.x1list(a)
n,quantilex1high=y.quantile(r)
z=y.changearticlelines(a,quantilex0low,quantilex1high)
h=y.changespeciallines(z,quantilex0low,quantilex1high)
h=y.deletepictureinfo(h)
h=y.deletetitle(h)
q=''
for i in h:
    q=q+i[4]
b=file()
v=b.Pdftotxtfile(q,'D:\\[Keep]\\Desktop\\project\\test.txt')


In [9]:
doc = fitz.open('D:\\[Keep]\\Desktop\\project\\pdfcollector\\try3.pdf')
for i in doc:
    r=i.get_text('blocks')
    print(i.get_pixmap())
    print(r,'\n')
#單位向素每個字大約6相素

Pixmap(DeviceRGB, IRect(0, 0, 596, 842), 0)
[(90.02400207519531, 44.609954833984375, 97.31570434570312, 54.5699577331543, '1 \n', 0, 0), (90.02400207519531, 769.9860229492188, 97.31570434570312, 792.1859741210938, '2 \n \n', 1, 0), (90.02400207519531, 76.0999755859375, 226.56289672851562, 88.0999755859375, '1111111111111111111111 \n', 2, 0)] 



In [10]:
doc = fitz.open('D:\\[Keep]\\Desktop\\project\\pdfcollector\\fb220309225809.pdf')
for annot in doc[10].widgets():
    print(annot)


In [11]:
p=file()
p.filenames()

['fb220309225809',
 'fb220309225908',
 'fb220309225926',
 'fb220309225936',
 'PSPICE Tutorial',
 'syllabus2022',
 'try',
 'try2',
 'try3',
 '碩士論文_盧毅_v1.7_20200819']

In [12]:
p.findpdfPutpath('fb220309225809')

'D:\\[Keep]\\Desktop\\project\\pdfcollector\\fb220309225809.pdf'

In [13]:
p.lenoftxt('D:\\[Keep]\\Desktop\\project\\total\\fb220309225809.txt')

130157

In [14]:
p.createdetail([1,2,3],[2,3,4],[5,6,7])

In [15]:
list('sdfaasdf')

['s', 'd', 'f', 'a', 'a', 's', 'd', 'f']

In [16]:
p.Pdftotxtfile('4564654','D:\\[Keep]\\Desktop\\project\\total\\fb220309225809.txt')

In [17]:
?p.Pdftotxtfile

In [18]:
?Demo.do_something

Object `Demo.do_something` not found.


In [19]:
import fitz
import pandas as pd 
doc = fitz.open('D:\\[Keep]\\Desktop\\project\\pdfcollector\\fb220309225809.pdf')
page1 = doc[3]
words = page1.get_text('blocks')
words

[(466.333251953125,
  787.962158203125,
  571.4832763671875,
  822.93017578125,
  'doi:10.6342/NTU202101736\n',
  0,
  0),
 (421.025634765625,
  70.8743896484375,
  524.525634765625,
  174.3743896484375,
  '<image: DeviceGray, width: 207, height: 207, bpc: 8>',
  1,
  1),
 (56.625, 34.869964599609375, 59.125, 48.209964752197266, ' \n', 2, 0),
 (292.70001220703125,
  781.844970703125,
  305.20001220703125,
  795.1849975585938,
  'III \n',
  3,
  0),
 (538.77001953125,
  793.2449340820312,
  541.27001953125,
  806.5849609375,
  ' \n',
  4,
  0),
 (277.5, 82.969970703125, 322.5, 109.64997100830078, '摘要 \n', 5, 0),
 (56.625,
  121.78398132324219,
  538.4620361328125,
  137.79197692871094,
  '自從 1979 年的美國以及 1990 年代的歐盟成員國家通過開放天空政策之後，造成航空產業的\n',
  6,
  0),
 (56.625,
  148.22064208984375,
  538.5059814453125,
  160.22064208984375,
  '市場競爭以及經營模式有顯著性的變遷。許多航空公司加入航空聯盟以增加市場的競爭性，\n',
  7,
  0),
 (56.625,
  171.640625,
  538.3529663085938,
  183.640625,
  '因此航空聯盟是由許多成員航空公司所組成，有共同的市場目標。機場的連結性在航空市場\n',

In [20]:
first_annots=[]
rec=page1.first_annot.rect
rec

AttributeError: 'NoneType' object has no attribute 'rect'

In [ ]:
[1]+[2]